In [31]:
# %pip install numpy opencv-python matplotlib
# %pip install keras

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pickle

DIRECTORY = os.path.join(os.getcwd(), 'dogscats', 'train')

CATEGORIES = ['cats', 'dogs', 'chicken']

In [32]:
data = []
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        new_arr = cv2.resize(arr, (60, 60))
        data.append([new_arr, label])

In [33]:
import random

random.shuffle(data)

In [34]:
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)
X = np.array(X)
y = np.array(y)

In [35]:
pickle.dump(X, open('X.pkl', 'wb'))
pickle.dump(y, open('y.pkl', 'wb'))

In [36]:
X = pickle.load(open('X.pkl', 'rb'))
y = pickle.load(open('y.pkl', 'rb'))

In [37]:
X = X/255
X = X.reshape(-1, 60, 60, 1)


In [38]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import BatchNormalization


In [39]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(60, 60, 1)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(len(CATEGORIES), activation='softmax'))

c:\Users\Tienn Vo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest",
    brightness_range=[0.8, 1.2],
    validation_split=0.3
)

In [42]:
# Tạo các generator cho huấn luyện và kiểm tra
train_generator = datagen.flow(X, y, batch_size=32, subset='training')  # Tập huấn luyện
validation_generator = datagen.flow(X, y, batch_size=32, subset='validation')  # Tập kiểm tra


In [43]:
# model.fit(train_generator, epochs=20, validation_data=validation_generator)
model.fit(X, y, epochs=20, validation_split=0.3)

Epoch 1/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 39s 65ms/step - accuracy: 0.4676 - loss: 0.9601 - val_accuracy: 0.5153 - val_loss: 0.8659
Epoch 2/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 38s 67ms/step - accuracy: 0.5800 - loss: 0.8172 - val_accuracy: 0.6607 - val_loss: 0.7197
Epoch 3/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 39s 69ms/step - accuracy: 0.6755 - loss: 0.6890 - val_accuracy: 0.7010 - val_loss: 0.6670
Epoch 4/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 39s 68ms/step - accuracy: 0.7368 - loss: 0.5921 - val_accuracy: 0.7338 - val_loss: 0.6143
Epoch 5/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 36s 62ms/step - accuracy: 0.7669 - loss: 0.5368 - val_accuracy: 0.7374 - val_loss: 0.5983
Epoch 6/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 36s 63ms/step - accuracy: 0.8137 - loss: 0.4493 - val_accuracy: 0.7554 - val_loss: 0.5903
Epoch 7/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 36s 63ms/step - accuracy: 0.8444 - loss: 0.3813 - val_accuracy: 0.7539 - val_loss: 0.5996
Epoch 8/20
571/571 ━━━━━━━━━━━━━━━━━━━━ 39s 69ms/step - accuracy: 0.8627 - loss: 0.3309 - 

In [44]:
model.save("model_trained.h5")